In [ ]:
import sys
sys.path.append('../../')
import opticalcoating as oc

# Инициализация дизайна

## Простейший дизайн без дисперсии

In [ ]:
# Данные
des_name = 'RAMP19'
d = [0.0, 11.312, 52.599, 40.663, 29.844, 48.71, 64.411, 22.164, 74.8, 46.238, 39.479, 39.97, 
     79.953, 36.046, 47.032, 49.216, 83.599, 51.294, 81.827, 52.024]
n_const = [1.52, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35, 1.46, 2.35]

# Создание дизайна
des = oc.Design(d=d, n_const=n_const)

# Сохранение простейшего дизайна в json
des.create_simple_json(name=des_name)

## Дизайн с дисперсией

### Из excel

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(filename = "28-layer polarizer for 1064 nm.xlsx")
ws = wb.active

des_name = 'POL28_Sch'

d_th = [0] + [x[0].value for x in ws['B402':'B429']]

print(f'Number of layers N = {len(d_th) - 1}')

subs, H, L = 'BK7', 'ZrO2', 'SiO2'

des_layers = [subs] + [H if i % 2 else L for i in range(1, len(d_th))]
        
print(f'Len des_layers = {len(des_layers)}')
print(f'des_layers = {des_layers[:4]}, ...')

mat_inf = {}
mat_inf[subs] = {'Table': {'wavelength': [x[0].value for x in ws['A26':'A125']],
                  'n': [x[0].value for x in ws['B26':'B125']]}}

mat_inf[H] = {'Table': {'wavelength': [x[0].value for x in ws['A146':'A216']],
                  'n': [x[0].value for x in ws['B146':'B216']]}}

mat_inf[L] = {'Table': {'wavelength': [x[0].value for x in ws['A229':'A389']],
                  'n': [x[0].value for x in ws['B229':'B389']]}}

for k in mat_inf: print(mat_inf[k]['Table']['wavelength'][0], mat_inf[k]['Table']['wavelength'][-1])

des_info = {'name': des_name,
           'thicknesses': d_th,
           'layers': des_layers,
           'mat_inf': mat_inf}

oc.Design(info=des_info).create_json()

# Свойства дизайна

### Загрузка дизайна

In [ ]:
# Если дизайн уже есть, то загружаем его так:
des_name = 'QWM20'
des = oc.Design(name=des_name)

In [ ]:
# Толщины
print('Толщины:', des.d)
# Показатели преломления
print('Показатели преломления:', des.n_const)

In [ ]:
des.thickness_bar(lang='ru')

In [ ]:
# Вычисление коэффициента пропускания T на длине волны 550 нм  со спектральной шириной щели 0.1 нм (абсолютное значение)
des.calc_flux(oc.Wave(600), q_subs=True, q_TR='R', width=0.1)

In [ ]:
# Спектральная характеристика (Performance)
des.spectral_plot(q_TR='R', wv_bnd=[380, 760], q_subs=False, lang='ru')
# Обе поляризации на одном графике
# des.spectral_plot(q_TR='T', wv_bnd=[1000, 1150], q_subs=False, lang='ru', polarisation='both', angle=55.6 * pi / 180, pic_ext='svg')

## Кривая мониторинга

In [ ]:
waves_1 = [oc.Wave(510) for _ in range(des.N + 1)]

msd = {}
msd[1] = oc.MonochromStrategyData.brute_force_calc(des, waves_1, q_TR='T', control_wv=550, q_subs=False)
msd[2] = oc.MonochromStrategyData.brute_force_calc(des, waves_1, q_TR='T', control_wv=550, q_subs=False, width=3)

### С нулевой шириной спектральной щели

In [ ]:
msd[1].monitoring_curve_plot(lang='ru')

### С шириной спектральной щели 3 нм

In [ ]:
msd[2].monitoring_curve_plot(lang='ru')

### Сравнение

In [ ]:
oc.MonochromStrategyData.monitoring_curve_plots([msd[k] for k in msd], lang='ru')

# Единичная симуляция

In [ ]:
from time import perf_counter, time

In [ ]:
# Алгоритмы прекращения напыления: 
# На выбор есть LM, QS, TM
term_algs = (des.N + 1) * ['QS']
# Длины волн, на которых ведется наблюдение
waves = [None] + des.N * [oc.Wave(550)]
# Скорости напыления
r_H, r_L = 0.4, 0.6
rates = [None] + [r_H if j % 2 else r_L for j in range(1, des.N + 1)]
# Ошибка в скорости напыления (стандартное отклонение)
rates_sigmas = [None] + [0.05 * r_H if j % 2 else 0.05 * r_L for j in range(1, des.N + 1)]

# Ошибка в измерении сигнала meas_sigmas=0.02 в абс. величинах (стандартное отклонение)
set_up_pars = oc.SetUpParameters(N=des.N, waves=waves, rates=rates, q_TR='T', meas_sigmas=0.005,
                                  rates_sigmas=rates_sigmas)

rnd_seed = int(1000000 * time())
task = {'des_th': des, 'term_algs': term_algs, 'set_up_pars': set_up_pars, 'rnd_seed': rnd_seed, 'save_M': True}

In [ ]:
start = perf_counter()
sim = oc.simulation(**task)
stop = perf_counter()
print(f'Время одной симуляции: {stop - start:0.3f} c')

In [ ]:
# Сохранение результата симуляции
sim.save()

In [ ]:
from matplotlib import rc
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 9))

_ = plt.bar(range(1, des.N + 1), sim.errors_d[1:])

In [ ]:
plt.figure(figsize=(16, 9))

for X, Y in zip(sim.time_list[1:], sim.flux_meas[1:]):
    plt.plot(X, Y)

# Сбор статистики

Как вычислять статистику смотри "research_scripts/python_scripts/Example.py"

## Визуализация ошибок

In [ ]:
stat_num = 1
oc.StatInfo.load(stat_num).rms_bar(lang='ru')

In [ ]:
from opticalcoating.statistics_info import error_norm_hist

error_norm_hist(stat_num)

## Анализ ошибок

### Задание таргета

In [ ]:
trg = oc.Target(name='RAMP19', 
          waves = [oc.Wave(x) for x in [400, 450, 500, 550, 600, 650, 700, 750, 800]], 
          flux_target = [0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0], 
          q_TR='T')
trg.save()

In [ ]:
c_vals = oc.timer(oc.ProcessedStatistics.calc)(des=des, target=oc.Target.from_json('RAMP19'), statistic_num=stat_num).save()
c_vals.save()

In [ ]:
print(f'Средняя норма ошибки {oc.mean_error_norm(stat_num):.2f} нм')
print(f'Коэф. самокомпенсации c = {oc.ProcessedStatistics.load(statistic_num=stat_num).c_value:.2f}')

## Визуализация

In [ ]:
oc.c_hist(oc.ProcessedStatistics.load(stat_num))

In [ ]:
from opticalcoating.performance_plot import performance_plot

performance_plot(statistic_num=stat_num, amount=5, is_best=False, wv_bnd=[450, 700], q_subs=False, ylim=(0, 100.5))

In [ ]:
from opticalcoating.performance_plot import performance_plot

performance_plot(statistic_num=stat_num, amount=5, is_best=True, wv_bnd=[450, 700], q_subs=False, ylim=(0, 100.5))